This notebook attempts to refactor the Segmentation code to convert it to a class that can be reused in other notebooks.

In [1]:
from transformers import pipeline
from PIL import Image
import numpy as np
import os

In [2]:
segmenter = pipeline(model="mattmdjaga/segformer_b2_clothes")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [22]:
def get_masked_image(segments, image, labels):
	mask_list = []
	for s in segments:
		if s['label'] in labels:
			mask_list.append(s['mask'])

	if len(mask_list) == 0:
		return None

	final_mask = np.zeros_like(mask_list[0])

	for mask in mask_list:
		final_mask += mask

	#final_mask = np.clip(final_mask, 0, 1)

	final_mask = Image.fromarray(final_mask)
	masked_image = image.copy()

	masked_image.putalpha(final_mask)

	return masked_image

In [23]:
def get_segmented_images(image_path, label_combinations):
	# Load the image
	#image = Image.open(image_path).convert("RGB")
	image = Image.open(image_path)
	
	# Perform segmentation
	segments = segmenter(image)

	segmented_images = {}

	for key, labels in label_combinations.items():
		#print(f"Getting segment {key} with labels: {labels}")
		masked_image = get_masked_image(segments, image, labels)

		if masked_image is not None:
			segmented_images[key] = masked_image

	return segmented_images


In [29]:
class SegmentationHelper:
	def __init__(self, data_dir, segmented_dir, label_combinations):
		self.data_dir = data_dir
		self.segmented_dir = segmented_dir
		self.label_combinations = label_combinations

		self.segmenter = pipeline(model="mattmdjaga/segformer_b2_clothes")

	def get_masked_image(self, segments, image, labels):
		mask_list = []
		for s in segments:
			if s['label'] in labels:
				mask_list.append(s['mask'])

		if len(mask_list) == 0:
			return None

		final_mask = np.zeros_like(mask_list[0])

		for mask in mask_list:
			final_mask += mask

		#final_mask = np.clip(final_mask, 0, 1)

		final_mask = Image.fromarray(final_mask)
		masked_image = image.copy()

		masked_image.putalpha(final_mask)

		return masked_image

	def get_segmented_images(self, image_path):
		image = Image.open(image_path)
		
		# Perform segmentation
		segments = self.segmenter(image)

		segmented_images = {}

		for key, labels in self.label_combinations.items():
			#print(f"Getting segment {key} with labels: {labels}")
			masked_image = self.get_masked_image(segments, image, labels)

			if masked_image is not None:
				segmented_images[key] = masked_image

		return segmented_images


In [30]:
seg_helper = SegmentationHelper(
    data_dir="../data/resized",
    segmented_dir="../data/segmented",
    label_combinations={
        "full": ["Upper-clothes", "Skirt", "Pants", "Dress", "Scarf"],
        "dress": ["Dress"],
        "upper": ["Upper-clothes"],
        "lower": ["Skirt", "Pants"],
    }
)

Device set to use cuda:0


In [ ]:
data_dir = "../data/resized"
segmented_dir = "../data/segmented"

label_combinations = {
	'full': ['Upper-clothes', 'Skirt', 'Pants', 'Dress', 'Scarf'],
	'dress': ['Dress'],
	'upper': ['Upper-clothes'],
	'lower': ['Skirt', 'Pants']
}

count = 0

for f in os.listdir(data_dir):
	image_path = os.path.join(data_dir, f)

	if not f.startswith('0540000'):
		continue

	if os.path.isfile(image_path):
		segmented_images = seg_helper.get_segmented_images(image_path)

		for key, masked_image in segmented_images.items():
			print(f"Segment '{key}' for image: {image_path}")
			masked_image.show()

			# # Save masked_image to file
			# output_dir = os.path.join(segmented_dir, f)
			# if not os.path.exists(output_dir):
			# 	os.makedirs(output_dir)

			# segmented_file_path = os.path.join(output_dir, f"{f}___{key}.png")
			# masked_image.save(segmented_file_path)
	
	count = count + 1

	if count > 3:
		break


